In [ ]:
# MetaData:

# id: unique identifier for each listing
# name: name of the listing
# host_id: unique identifier for the host
# host_name: name of the host
# neighbourhood_group: grouping of neighbourhoods
# neighbourhood: name of the neighbourhood
# latitude: latitude coordinate of the listing
# longitude: longitude coordinate of the listing
# room_type: type of room in the listing (e.g., Entire home/apt, Private room)
# price: price per night for the listing
# minimum_nights: minimum number of nights required for booking
# number_of_reviews: total number of reviews for the listing
# last_review: date of the last review
# reviews_per_month: average number of reviews per month
# calculated_host_listings_count: total number of listings by the host
# availability_365: number of days the listing is available within the next 365 days

# Example 5 rows of the dataset
# | id   | name             | host_id | host_name | neighbourho| neighbourhood     | latitude | longitude | room_type  | price | mini...| number| last_review | review...| calc...| availability_365 |
# |------|------------------|---------|-----------|------------|-------------------|----------|-----------|------------|-------|--------|-------|-------------|----------|--------|------------------|
# | 2015 | Berlin-Mitte...  | 2217    | Ian       | Mitte      | Brunnenstr. Süd   | 52.5345  | 13.4026   | Entire...  | 60    | 4      | 118   | 28-10-18    | 3.76     | 4      | 141              |
# | 2695 | Prenzlauer...    | 2986    | Michael   | Pankow     | Prenzlauer Berg...| 52.5485  | 13.4046   | Private... | 17    | 2      | 6     | 1/10/2018   | 1.42     | 1      | 0                |
# | 3176 | Fabulous...      | 3718    | Britta    | Pankow     | Prenzlauer Berg...| 52.535   | 13.4176   | Entire...  | 90    | 62     | 143   | 20-03-17    | 1.25     | 1      | 220              |
# | 3309 | BerlinSpot...    | 4108    | Jana      | Tempelhof  | Schöneberg-Nord   | 52.4989  | 13.3491   | Private... | 26    | 5      | 25    | 16-08-18    | 0.39     | 1      | 297              |
# | 7071 | BrightRoom...    | 17391   | Bright    | Pankow     | Helmholtzplatz    | 52.5432  | 13.4151   | Private... | 42    | 2      | 197   | 4/11/2018   | 1.75     | 1      | 26               |


In [ ]:
pip install cassandra-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 34.8 MB/s eta 0:00:00


In [ ]:

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config= {
        'secure_connect_bundle': 'secure-connect-mydatabase.zip' # replace <</PATH/TO/>> with the path where your downloaded bundle was downloaded (make sure to place this python file in the same place as well)
}
auth_provider = PlainTextAuthProvider('<<CLIENT ID>>', '<<CLIENT SECRET>>') # replace <<CLIENT ID>> and <<CLIENT SECRET>> with the ClientID and Client Secret from your generated token
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
session.set_keyspace('airbnb') # replace <<KEYSPACENAME>> with the name of the keyspace you created in DataStax


In [ ]:
# -✔️-- Describing a keyspace
row = cluster.metadata.keyspaces['airbnbkeyspace'] # replace <<KEYSPACENAME>> with the name of the keyspace you created in DataStax

from pprint import pprint
pprint(vars(row))

# --RESULT--
{'aggregates': {},
 'durable_writes': True,
 'functions': {},
 'graph_engine': None,
 'indexes': {},
 'name': 'airbnbkeyspace',
 'replication_strategy': <cassandra.metadata.NetworkTopologyStrategy object at 0x0000016022846690>,
 'tables': {'berlinairbnb': <cassandra.metadata.TableMetadataV3 object at 0x0000016023B8C1D0>},
 'user_types': {'coordinates': <cassandra.metadata.UserType object at 0x0000016023B8C0D0>},
 'views': {}}

In [ ]:
# DROPPING THE TABLE
session.execute("DROP TABLE IF EXISTS airbnbkeyspace.berlinairbnb;")

In [ ]:
# DROP THE UDT
session.execute("DROP TYPE IF EXISTS airbnbkeyspace.coordinates;")

In [ ]:
# -✔️-- Create UDT for the coordinates (longitude and latitude)

session.execute("create type if not exists coordinates ( longitude float, latitude float );")

In [ ]:
# -✔️- Reading listings from CSV into dataframe

import pandas as pd

df = pd.read_csv('C:/Users/asus/Downloads/listings.csv', encoding= 'unicode_escape') # edit path accordingly to read from where the moviestv.csv file is
df = df.drop(columns=['Unnamed: 0'])
df = df.rename(columns={'name':"title"})

i,j = df.shape

In [ ]:
# -✔️- Inserting the listings into DB (Creating UDT for Coordinates, Creating Table + Inserts)

session.execute("create type long_lat ( longitude float, latitude float );")

session.execute("""
    create table if not exists airbnb.berlin (
        id int primary key,
        listing_title text,
        host_id int,
        host_name text,
        neighbourhood text,
        neighbourhood_group text,
        coordinates long_lat,
        room_type text,
        price int,
        minimum_nights int,
        availability int
 	);
""")

num = 0
import math

for x in range(0,1000):

    listing_id = int(df.loc[x].id) if not math.isnan(df.loc[x].id) else "null"
    listing_title = "null" if str(df.loc[x].title)=="nan" else df.loc[x].title
    host_id = int(df.loc[x].host_id) if not math.isnan(df.loc[x].host_id) else "null"
    host_name = df.loc[x].host_name if str(df.loc[x].host_name).strip() else "null"
    neighbourhood = df.loc[x].neighbourhood if str(df.loc[x].neighbourhood).strip() else "null"
    neighbourhood_group = df.loc[x].neighbourhood_group if str(df.loc[x].neighbourhood_group).strip() else "null"
    room_type = df.loc[x].room_type if str(df.loc[x].room_type).strip() else "null"
    price = int(df.loc[x].price) if not math.isnan(df.loc[x].price) else "null"
    minimum_nights = int(df.loc[x].minimum_nights) if not math.isnan(df.loc[x].minimum_nights) else "null"
    availability = int(df.loc[x].availability_365) if not math.isnan(df.loc[x].availability_365) else "null"

    if "'" not in str(listing_title) and "'" not in str(host_name):
        # session.execute("insert into airbnb.berlin(id,listing_title,host_id,host_name,neighbourhood,neighbourhood_group,coordinates,room_type,price,minimum_nights,availability) values (%d,'%s',%d,'%s','%s','%s',{longitude:%d,latitude:%d},'%s',%d,%d,%d);" % (listing_id,listing_title,host_id,host_name,neighbourhood,neighbourhood_group,df.loc[x].longitude,df.loc[x].latitude,room_type,price,minimum_nights,availability))

        # --OR--

        session.execute(
            "insert into airbnb.berlin(id,listing_title,host_id,host_name,neighbourhood,neighbourhood_group,coordinates,room_type,price,minimum_nights,availability) "
            "values (%d,'%s',%d,'%s','%s','%s',{longitude:%d,latitude:%d},'%s',%d,%d,%d);" % (
                listing_id, listing_title, host_id, host_name, neighbourhood, neighbourhood_group,
                df.loc[x].longitude, df.loc[x].latitude, room_type, price, minimum_nights, availability
            )
        )

        num = num + 1

print(num,"listings were inserted")

rows = session.execute("select * from airbnb.berlin limit 10;") # to make sure entries where inserted + correctly
for row in rows:
    print(row)

In [ ]:
# -✔️- Removing listings with missing price quota (need to index price first) ----------------------------------------------------------

session.execute("create index on airbnb.berlin(price);")

rows = session.execute("select * from airbnb.berlin where price=0 OR price IS NULL;")
num = 0
ids = []
for row in rows:
    num = num+1
    ids = ids + [row[0]]
    print(row)

for n in ids:
    session.execute("delete from airbnb.berlin where id='{0}';".format(n))

print(num,"rows deleted")

In [ ]:
# -✔️- Optimizing search results - on neighbourhood_group (showing execution timings before and after optimization) ----------------------------------------------------------

session.execute("create index on airbnb.berlin(neighbourhood_group);")

rows = session.execute("select * from airbnb.berlin where neighbourhood_group='Mitte';")
for row in rows:
    print(row)

In [ ]:
# -✔️⚠️- Finding the top 3 private rooms with minimum amount to pay (adding new column duration_rent + updating it) + displaying the listing_id, cost, duration rent, minimum nights & neighbourhood

session.execute("alter table airbnb.berlin add duration_rent int;")

session.execute("create index on airbnb.berlin(room_type);")

rows = session.execute("select id,price,minimum_nights from airbnb.berlin where room_type='Private room';")
num = 0

for row in rows:
    if row[1] is not None and row[2] is not None:
        num = num+1
        dr = int(row[1])*int(row[2])
        session.execute("update airbnb.berlin set duration_rent={0} where id={1};".format(dr,int(row[0])))
        print(num)

print(num,"rows updated")

"""

session.execute("create index on airbnb.berlin(duration_rent);")

result = session.execute("select id,duration_rent,minimum_nights,neighbourhood from airbnb.berlin where room_type='Private room' order by duration_rent ASC limit 3;")
num = 0

for l in result:
    print("Listing",l[0],"costs $",l[1],"for",l[2],"day(s) and is in",l[3])

"""

######### WILL NOT BE POSSIBLE WITHOUT DROPPING TABLE THEN MAKING DURATION_RENT AS CLUSTERING KEY + ORDER BY ASC #########

# Create a **Materialzed View**


Materialized views allow you to denormalize your data and create a separate table that is indexed and optimized for specific queries. This can be useful for querying on non-primary key columns.


- This solves the problem of us not being able to query on non-primary keys

In [ ]:
# Create the main table
session.execute(
    """
    CREATE TABLE IF NOT EXISTS people (
        person_id UUID PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        age INT
    )
    """
)

# Insert some dummy data
session.execute(
    """
    INSERT INTO people (person_id, first_name, last_name, age) VALUES
        (uuid(), 'John', 'Doe', 30),
        (uuid(), 'Jane', 'Smith', 25),
        (uuid(), 'Alice', 'Johnson', 35)
    """
)

# Create a materialized view to query on the "age" column
session.execute(
    """
    CREATE MATERIALIZED VIEW IF NOT EXISTS people_by_age AS
        SELECT person_id, first_name, last_name, age
        FROM people
        WHERE age IS NOT NULL
        PRIMARY KEY (age, person_id)
    """
)

# Query the materialized view to get all people with age 30
rows = session.execute("SELECT * FROM people_by_age WHERE age = 30;")
for row in rows:
    print(row)